In [1]:
username = 'MarcelloCeresini'
repository = 'QuestionAnswering'

# COLAB ONLY CELLS
try:
    import google.colab
    IN_COLAB = True
    !pip3 install transformers
    !git clone https://www.github.com/{username}/{repository}.git
    #from google.colab import drive
    #drive.mount('/content/drive/')
    %cd /content/QuestionAnswering/src
    using_TPU = True    # If we are running this notebook on Colab, use a TPU
except:
    IN_COLAB = False
    using_TPU = False   # If you're not on Colab you probably won't have access to a TPU

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 77 kB 4.6 MB/s 
     |████████████████████████████████| 6.6 MB 13.0 MB/s 
     |████████████████████████████████| 895 kB 46.1 MB/s 
     |████████████████████████████████| 596 kB 22.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'QuestionAnswering'...
remote: Enumerating objects: 918, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 918 (delta 57), reused 31 (delta 28), pack-reused 830
Receiving objects: 100% (918/918), 28.91 MiB | 8.26 MiB/s, done.
Resolving deltas: 100% (554/554), done.
Checking out files: 100% (48/48), done.
/content/QuestionAnswering/src


# Description

In this notebook, we will try to implement the architecture detailed in [Dense Passage Retrieval for Open-Domain Question Answering](https://arxiv.org/pdf/2004.04906.pdf). 

The idea is that we have a corpus of documents $C = {p_1, p_2, \dots, p_M}$ where each passage $p_i$ can be viewed as a sequence of tokens $w_1^{(i)}, w_2^{(i)}, \dots, w_{|p_i|}^{(i)}$ and given a question $q$ we want to find the sequence of tokens $w_s^{(i)}, w_{s+1}^{(i)}, \dots, w_{e}^{(i)}$ from one of the passage $i$ that can answer the question.

In order to find the passage $i$ we need an efficient **Retriever** (i.e. a function $R: (q, C) \rightarrow C_F$ where $C_F$ is a very small set of $k$ documents that have a high correlation with the query.)

In the Tf-Idf example, the retriever was simply a function that returned the top 5 scores obtained by computing the vector cosine similarity between the query and all other documents. The problem with this approach is that it is not very efficient. Tf-Idf is a **sparse** document/query representation, thus computing a multitude of dot products between these very long vectors can be expensive.

The paper cited above proposes a **dense** representation instead. It uses a Dense Encoder $E_P$ which maps all paragraphs to $d$-dimensional vectors. These vectors are stored in a database so that they can be efficiently retrieved. 

At run-time, another Dense Encoder is used $E_Q$ which maps the input question to a vector with the same dimensionality $d$. Then, a similarity score is computed between the two representations:

$sim(p,q) = E_Q(q)^\intercal E_P(p)$

In the paper, $E_Q$ and $E_P$ are two independent BERT transformers and the $d$-dimensional vector is the **output at the $\texttt{[CLS]}$ token** (so, $d = 768$).
- This leaves open the possibility to use a larger dimensionality (eg. concatenating the output at multiple blocks like we did for the QA task).

The $d$-dimensional representations of the $M$ passages are indexed using [FAISS](https://github.com/facebookresearch/faiss), an efficient, open-source library for similarity search and clustering of dense vectors developed at Facebook AI. At run-time, we simply compute $v_q = E_Q(q)$ and retrieve the top $k$ passages with embeddings closest to $v_q$.

In this case, training the network means solving a **metric learning** problem: the two BERT networks need to learn an **effective vector space** such that relevant pairs of questions and passages are close, while irrelevant pairs are placed further away. In this problem we usually build a **training instance $D$** as ${(q_i, p_i^+, p_{i,1}^-, p_{i,2}^-, \dots, p_{i,n}^-)}^m_{i=1}$, where question $q$ is paired with a relevant (positive) passage $p_i^+$ and $n$ irrelevant (negative) passages. Then, the loss function is the negative log-likelihood of the positive passage:

$L(q_i, p_i^+, p_{i,1}^-, p_{i,2}^-, \dots, p_{i,n}^-) = -\log\frac{e^{sim(q_i, p_i^+)}}{e^{sim(q_i, p_i^+)} + \sum_{j=1}^n e^{sim(q_i, p_{i,j}^-)}}$

It's easy to find the positive paragraph, but choosing the negatives is quite important. In particular, the paper proposes different ways for sampling the negatives:
- Random: a negative is any random passage in the corpus
- TF-IDF (The paper uses a variant, BM25): the negatives are the top passages (not containing the answer) returned by a TF-IDF search
- Gold: the negatives are positives for other questions in the mini-batch. For the researchers, this is the best negative-mining option, because it's the most efficient and also it makes a batch a complete unit of learning (we learn the relationship that each question in the batch has with the other paragraphs).

The Gold method allows the **in-batch negatives** technique: assuming to have a batch size of $B$, then we collect two $B \times d$ matrices (one for questions, one for their positive paragraphs). Then, we compute $S = QP^\intercal$ which is a $B \times B$ matrix of **similarity scored** between each question and paragraph. This matrix can directly be used for training: any ($q_i, p_j$) pais where $i = j$ is considered to be a positive example, while it's negative otherwise. In total there will be $B$ training instances per batch, each with $B-1$ negative passages. 

# Configuration

## Imports

In [2]:
import os
import numpy as np
import random
from tqdm import tqdm
from functools import partial
tqdm = partial(tqdm, position=0, leave=True)
import tensorflow as tf
from transformers import BertTokenizerFast, DistilBertTokenizerFast, \
                         TFBertModel, TFDistilBertModel
import utils

RANDOM_SEED = 42
BERT_DIMENSIONALITY = 768

np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

### TPU check
The training could be made faster if we use the cloud GPUs offered by Google on Google Colab. Since TPUs require manual intialization and other oddities, we check multiple times throughout the notebook what kind of hardware we are running the code on.
using_TPU = True


In [3]:
if using_TPU:
    try: 
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
        tf.config.experimental_connect_to_cluster(resolver)
        # This is the TPU initialization code that has to be at the beginning.
        tf.tpu.experimental.initialize_tpu_system(resolver)
        print("All devices: ", tf.config.list_logical_devices('TPU'))
        strategy = tf.distribute.TPUStrategy(resolver)
    except:
        print("TPUs are not available, setting flag 'using_TPU' to False.")
        using_TPU = False
else:
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.8.96.194:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.8.96.194:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [4]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive/')
    checkpoint_dir = '/content/drive/My Drive/Uni/Magistrale/NLP/Project/weights/training_dpr/'
    datasets_dir = '/content/drive/My Drive/Uni/Magistrale/NLP/Project/datasets/dpr/'
else:
    # Create the folder where we'll save the weights of the model
    checkpoint_dir = os.path.join("data", "training_dpr")
    datasets_dir = os.path.join("data", "training_dpr", "dataset")

os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(datasets_dir, exist_ok=True)

Mounted at /content/drive/


## Allowed operations

In [5]:
OVERWRITE_DATASETS = False      # Best to be run on GPU, takes about half an hour.
                                # It saves a new copy of the pre-tokenized
                                # question-paragraph pair on Google Drive.
                                # The copy should then be uploaded to GCloud into
                                # the bucket allocated for the project to be used by
                                # TPUs.

DO_TRAINING = False             # Best to be run on TPU, takes about 2 hours.
                                # It trains the model and saves a copy of the initial
                                # and last weights on Google Drive. When run on GPU
                                # It also saves checkpoints and Tensorboard data,
                                # but it takes over 4 hours to do a single epoch,
                                # so it's impossible to run it on Colab.

OVERRIDE_REPRESENTATIONS = False # Best to be run on GPU, it takes about 30 minutes.
                                 # It produces representations of all paragraphs
                                 # using the trained model_p as a large NumPy array
                                 # that is stored on Google Drive at the end.

RUN_ACCURACY_ANALYSIS = False   # Best to be run on GPU, takes about an hour
                                # It analyzes the paragraph retrieval capability
                                # of the model with top-1 and top-5 accuracy.

DO_QA_TRAINING = True          # Best to be run on TPU, takes more than 2 hours.
                                # See DO_TRAINING.

## Variables

We define all the paths.

In [6]:
ROOT_PATH = os.path.dirname(os.getcwd())
TRAINING_FILE = os.path.join(ROOT_PATH, 'data', 'training_set.json')
VALIDATION_FILE = os.path.join(ROOT_PATH, 'data', 'validation_set.json')

We collect the training and validation questions and paragraphs into different lists.

In [7]:
train_dict = utils.read_question_set(TRAINING_FILE)
val_dict = utils.read_question_set(VALIDATION_FILE)

def get_questions_and_paragraphs_from_dataset(dataset):
    questions = [{
            'qas': qas,
            'context_id': (i,j)    # We also track the question's original context and paragraph indices so to have a ground truth
        }
        for i in range(len(dataset['data']))
        for j, para in enumerate(dataset['data'][i]['paragraphs'])
        for qas in para['qas']
    ]

    paragraphs = [{
            'context': para['context'],
            'context_id': (i,j)
        }
        for i in range(len(dataset['data']))
        for j, para in enumerate(dataset['data'][i]['paragraphs'])
    ]

    for i in range(len(questions)):
        questions[i]['global_id'] = i
    for i in range(len(paragraphs)):
        paragraphs[i]['global_id'] = i

    return questions, paragraphs

train_questions, train_paragraphs = get_questions_and_paragraphs_from_dataset(train_dict)
val_questions, val_paragraphs = get_questions_and_paragraphs_from_dataset(val_dict)

We create the two different DistilBert models for encoding and test them on a random question/paragraph.

In [8]:
tokenizer_distilbert = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model_q, model_p = TFDistilBertModel.from_pretrained('distilbert-base-uncased'), \
                   TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_projector', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.
Some layers from the model checkpoint at distilbert-base-uncased were no

In [9]:
test_question = train_questions[0]['qas']['question']
print(f"Testing on a simple question. \nQuestion: {test_question}")
inputs_test = tokenizer_distilbert(test_question, return_tensors="tf")
outputs = model_q(inputs_test)

# As a representation of the token we use the last hidden state at the [CLS] token (the first one)
last_hidden_states = outputs.last_hidden_state
test_q_repr = last_hidden_states[0,0,:]
print(f"Representation dimensionality: {test_q_repr.shape}")

Testing on a simple question. 
Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Representation dimensionality: (768,)


# Encoders Training

First of all, we need to train our models. To do that, we need to create a dataset that feeds batches of questions and positive and negative paragraphs to a model, which is used to compute the representations, then the similarities and to correct the learnt distributions from the encoder models.

## Dataset creation

For the dataset, we use the `tf.data.Dataset` API. In particular, we first create the dataset in a `.proto` file which contains all the information in a byte format. The file is then uploaded on Google Cloud, where it can easily be accessed all future times. This is the only working way to have a large dataset accessible from the TPU.

In [10]:
import math
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import Sequence
from typing import List, Union
from tqdm import tqdm
from typing import Dict, Tuple

BATCH_SIZE = 8 if not using_TPU else 64

def get_paragraph_from_question(qas, dataset):
    i,j = qas['context_id']
    return dataset['data'][i]['paragraphs'][j]

def get_paragraph_global_id_from_question(question, paragraphs):
    for p in paragraphs:
        if p['context_id'] == question['context_id']:
            return p['global_id']

def pre_tokenize_data(questions, dataset, tokenizer):
    tokenized_questions = [
        dict(tokenizer(questions[i]['qas']['question'], 
            max_length = 512, truncation = True, 
            padding = 'max_length'))
    for i in tqdm(range(len(questions)))]
    tokenized_paragraphs = [
        dict(tokenizer(get_paragraph_from_question(
                    questions[i], dataset
                )['context'], max_length = 512, 
            truncation = True, padding = 'max_length',
            return_offsets_mapping = True))
    for i in tqdm(range(len(questions)))]
    return tokenized_questions, tokenized_paragraphs

def find_start_end_token_one_hot_encoded(
    answers: Dict, 
    offsets: List[Tuple[int]]) -> Dict:
    '''
    This function returns the starting and ending token of the answer, 
    already one hot encoded and ready for binary crossentropy.
    Inputs:
        - answers: `List[Dict]` --> for each question, a list of answers.
            Each answer contains:
            - `answer_start`: the index of the starting character
            - `text`: the text of the answer, that we exploit through the 
                number of chars that it contains
        - offsets: `List[Tuple[int]]` --> the tokenizer from HuggingFace 
            transforms the paragraph into a sequence of tokens. 
            Offsets keeps track of the character start and end indexes for each token.
   
    Output:
        - result: `Dict` --> each key contains only one array, the one-hot 
            encoded version of, respectively, the start and end token of 
            the answer in the sentence (question+context)
    '''
    result = {
        "out_S": np.zeros(len(offsets), dtype=np.int32),
        "out_E": np.zeros(len(offsets), dtype=np.int32)
    } 
    for answer in answers:
        starting_char = answer['answer_start']
        answer_len = len(answer['text'])
        # We skip the first token, [CLS], that has (0,0) as a tuple
        for i in range(1, len(offsets)):
            # Check if starting char is within the indexes
            if (starting_char >= offsets[i][0]) and \
                (starting_char <= offsets[i][1]):
                result["out_S"][i] += 1
            # If the ending char (starting + length -1) is in the interval, 
            # same as above.
            if (starting_char + answer_len - 1 >= offsets[i][0]) and \
                (starting_char + answer_len - 1 < offsets[i][1]):
                result["out_E"][i] += 1
                break
    return result

def decode_fn(record_bytes):
  example = tf.io.parse_single_example(
      # Data
      record_bytes,
      # Schema
      {"question__input_ids": tf.io.FixedLenFeature(shape=(512,), dtype=tf.int64),
       "question__attention_mask": tf.io.FixedLenFeature(shape=(512,), dtype=tf.int64),
       "question__id": tf.io.FixedLenFeature(shape=(), dtype=tf.string),
       "answer__out_s": tf.io.FixedLenFeature(shape=(512,), dtype=tf.int64),
       "answer__out_e": tf.io.FixedLenFeature(shape=(512,), dtype=tf.int64),
       "paragraph__input_ids": tf.io.FixedLenFeature(shape=(512,), dtype=tf.int64),
       "paragraph__attention_mask": tf.io.FixedLenFeature(shape=(512,), dtype=tf.int64),
       "paragraph__text": tf.io.FixedLenFeature(shape=(), dtype=tf.string),
       "paragraph__tokens_s": tf.io.FixedLenFeature(shape=(512,), dtype=tf.int64),
       "paragraph__tokens_e": tf.io.FixedLenFeature(shape=(512,), dtype=tf.int64),
       "paragraph__index": tf.io.FixedLenFeature(shape=(), dtype=tf.int64)})
  return {
      "questions": {'input_ids': example['question__input_ids'],
                    'attention_mask': example['question__attention_mask'],
                    'id': example['question__id']},
      "answers":   {'out_s': example['answer__out_s'],
                    'out_e': example['answer__out_e']},
      "paragraphs":{'input_ids': example['paragraph__input_ids'],
                    'attention_mask': example['paragraph__attention_mask'],
                    'tokens_s': example['paragraph__tokens_s'],
                    'tokens_e': example['paragraph__tokens_e'],
                    'text': example['paragraph__text'],
                    'index': example['paragraph__index']}
  }

def create_dataset_from_records(questions, paragraphs, dataset, tokenizer, fn, training=True):
    # Pre-tokenize and write dataset on disk
    filename = f'{fn}.proto'
    fn_type = filename.split(os.sep)[-1].replace('.proto','')
    gcs_filename = os.path.join('gs://volpepe-nlp-project-squad-datasets', f'{fn_type}.proto')
    if not os.path.exists(filename) or OVERWRITE_DATASETS:
        print("Pre-tokenizing data...")
        tok_questions, tok_paragraphs = pre_tokenize_data(questions, dataset, tokenizer)
        assert len(tok_questions) == len(tok_paragraphs), "Error while pre-tokenizing dataset"
        print("Preprocessing answers...")
        answer_tokens = [find_start_end_token_one_hot_encoded(
            questions[i]['qas']['answers'], tok_paragraphs[i]['offset_mapping'])
        for i in tqdm(range(len(questions)))]
        print("Saving dataset on disk...")
        with tf.io.TFRecordWriter(filename) as file_writer:
            for i in tqdm(range(len(tok_questions))):
                record_bytes = tf.train.Example(features=tf.train.Features(feature={
                    "question__input_ids": tf.train.Feature(int64_list=tf.train.Int64List(
                            value=tok_questions[i]["input_ids"])),
                    "question__attention_mask": tf.train.Feature(int64_list=tf.train.Int64List(
                            value=tok_questions[i]["attention_mask"])),
                    "question__id": tf.train.Feature(bytes_list=tf.train.BytesList(
                        value=[questions[i]['qas']['id'].encode('utf-8')])),
                    "answer__out_s": tf.train.Feature(int64_list=tf.train.Int64List(
                        value=answer_tokens[i]["out_S"])),
                    "answer__out_e": tf.train.Feature(int64_list=tf.train.Int64List(
                        value=answer_tokens[i]["out_E"])),
                    "paragraph__input_ids": tf.train.Feature(int64_list=tf.train.Int64List(
                        value=tok_paragraphs[i]["input_ids"])),
                    "paragraph__attention_mask": tf.train.Feature(int64_list=tf.train.Int64List(
                        value=tok_paragraphs[i]["attention_mask"])),
                    "paragraph__tokens_s": tf.train.Feature(int64_list=tf.train.Int64List(
                            value=[x[0] for x in tok_paragraphs[i]["offset_mapping"]])),
                    "paragraph__tokens_e": tf.train.Feature(int64_list=tf.train.Int64List(
                            value=[x[1] for x in tok_paragraphs[i]["offset_mapping"]])), 
                    "paragraph__text": tf.train.Feature(bytes_list=tf.train.BytesList(
                        value=[get_paragraph_from_question(
                            questions[i], dataset
                        )['context'].encode('utf-8')])),
                    "paragraph__index": tf.train.Feature(int64_list=tf.train.Int64List(
                        value=[get_paragraph_global_id_from_question(questions[i], paragraphs)]))
                    })).SerializeToString()
                file_writer.write(record_bytes)
        print("Upload the dataset on Google Cloud and re-run the function")
        return None
    print(f"Loading {fn_type} dataset from GCS ({gcs_filename}).")
    # Return it as processed dataset
    dataset = tf.data.TFRecordDataset([gcs_filename]).map(decode_fn)
    dataset = dataset.cache()
    dataset = dataset.shuffle(10000)
    if training:
        dataset = dataset.repeat()
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    dataset = dataset.apply(tf.data.experimental.assert_cardinality(len(questions)))
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

### CREATE DATASETS ###
# Valid for both TPU and GPU
dataset_train = create_dataset_from_records(train_questions, train_paragraphs, train_dict, tokenizer_distilbert, 
                                            os.path.join(datasets_dir, 'train'))
dataset_val = create_dataset_from_records(val_questions, val_paragraphs, val_dict, tokenizer_distilbert,
                                            os.path.join(datasets_dir, 'val'), training=False)

Loading train dataset from GCS (gs://volpepe-nlp-project-squad-datasets/train.proto).
Loading val dataset from GCS (gs://volpepe-nlp-project-squad-datasets/val.proto).


## Training pipeline

First of all, we need a layer that takes as input the dictionary containing the tokenized questions and answers and returns their compact representations.

In [11]:
class DenseEncoder(layers.Layer):
    def __init__(self, model_q, model_p):
        super().__init__()
        self.model_q = model_q  # Dense encoder for questions
        self.model_p = model_p  # Dense encoder for paragraphs
    
    def call(self, inputs, training=False):
        qs = {
            'input_ids': inputs['questions']['input_ids'],
            'attention_mask': inputs['questions']['attention_mask']
        }
        q_repr = self.model_q(qs).last_hidden_state[:,0,:]
        if training:
            # Input contains the questions and paragraphs encoding
            ps = qs = {
                'input_ids': inputs['paragraphs']['input_ids'],
                'attention_mask': inputs['paragraphs']['attention_mask']
            }
            p_repr = self.model_p(ps).last_hidden_state[:,0,:]
            return q_repr, p_repr
        else:
            return q_repr

# Small test for the layer
class TestDenseEncoderModel(keras.Model):
    def __init__(self, model_q, model_p):
        super().__init__()
        self.enc = DenseEncoder(model_q, model_p)

    def call(self, inputs, training=False):
        return self.enc(inputs, training=training)

test_model = TestDenseEncoderModel(model_q, model_p)
q_repr, p_repr = test_model(next(dataset_train.take(1).as_numpy_iterator()), training=True)
print(f"Output shape when in training mode: {q_repr.shape}, {p_repr.shape}")
q_repr_2 = test_model(next(dataset_train.take(1).as_numpy_iterator()), training=False)
print(f"Output shape when in testing mode: {q_repr_2.shape}")
q = tokenizer_distilbert(
    train_questions[0]['qas']['question'], max_length = 512, 
    truncation = True, padding = 'max_length', return_tensors="tf")
q_repr_3 = test_model({'questions': q})
print(f"Output shape when dealing with a single question: {q_repr_3.shape}")

Output shape when in training mode: (64, 768), (64, 768)
Output shape when in testing mode: (64, 768)
Output shape when dealing with a single question: (1, 768)


Once we have the representations, we should compute the similarities, thus obtaining a a full mini-batch of positive-negative examples. 

In [12]:
# Create the similarity matrix
S = tf.tensordot(q_repr, tf.transpose(p_repr), axes=1)
S.shape

TensorShape([64, 64])

This similarity matrix has the following meaning:
- Rows represent questions.
- Each row contains the similarity that the respective question has with the 16 paragraphs (one of them is the positive one, the others are negative)

In the paper, they refer to the loss as a *minimization of the negative log-likelihood of the positive passage*: what it really means is that we need to transform similarities to probabilities and use a categorical cross-entropy loss, where labels are the row index (which is also the column index in that row for the positive passage)

In [13]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True
)
loss(y_true=tf.range(BATCH_SIZE), y_pred=S).numpy()

8.03569

The loss seems to be quite high for this batch. We can study it with a confusion matrix.

In [14]:
from sklearn.metrics import ConfusionMatrixDisplay

if not using_TPU: # Otherwise the batch size is HUGE
    S_arr = S.numpy()
    ConfusionMatrixDisplay.from_predictions(
        y_true=np.arange(BATCH_SIZE), y_pred=np.argmax(S_arr, axis=1))

Indeed, ideally the predictions should be on the diagonal. This means that the "default" space for this metric learning problem is not that good. We are ready to learn a new representation distribution.

## Model definition

In [15]:
class DeepQPEncoder(keras.Model):

    def __init__(self, model_q, model_p):
        super().__init__()
        self.enc = DenseEncoder(model_q, model_p)

    def call(self, inputs, training=False):
        if training:
            # For training we return the similarity matrix
            repr_q, repr_p = self.enc(inputs, training=training)
            S = tf.tensordot(repr_q, tf.transpose(repr_p), axes=1)
            return S
        else:
            # In other cases, we return the representation of the question(s)
            repr_q = self.enc(inputs, training=training)            
            return repr_q

    def train_step(self, data):
        x = data
        y = tf.range(tf.shape(x['questions']['input_ids'])[0])
        with tf.GradientTape() as tape:
            # Obtain similarities
            S = self(x, training=True)
            # Obtain loss value
            loss = self.compiled_loss(y, S)
        # Construct gradients and apply them through the optimizer
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        # Update and return metrics (specifically the one for the loss value).
        self.compiled_metrics.update_state(y, S)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        x = data
        y = tf.range(tf.shape(x['questions']['input_ids'])[0])
        S = self(x, training=True) # We are not really training, but we have to obtain S
        self.compiled_loss(y, S)
        self.compiled_metrics.update_state(y, S)
        return {m.name: m.result() for m in self.metrics}

def create_model(sample, freeze_layers_up_to=5):
    print("Creating BERT models...")
    model_q, model_p =  TFDistilBertModel.from_pretrained('distilbert-base-uncased'), \
                        TFDistilBertModel.from_pretrained('distilbert-base-uncased')

    # Freeze layers 
    for i in range(freeze_layers_up_to): # layers 0 to variable are frozen, successive layers learn
        model_q.distilbert.transformer.layer[i].trainable = False
        model_p.distilbert.transformer.layer[i].trainable = False
    
    print("Creating Deep Encoder...")
    model = DeepQPEncoder(model_q, model_p)

    print("Compiling...")
    # Compile the model and loss
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=3e-6),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy()]
    )

    print("Testing on some data...")
    # Pass one batch of data to build the model
    model(sample)

    # Return the model
    print("Model created!")
    return model

## Training procedure

Define utility variables and saving paths.

In [16]:
EPOCHS = 100
PATIENCE = 3

Before training, we check if we're using a TPU, in order to create the model within the scope of the strategy.

Then, we train the model.

In [17]:
import datetime

if using_TPU:
    # TPU requires to create the model within the scope of the distributed strategy
    # we're using.
    with strategy.scope():
        model = create_model(sample=next(dataset_train.take(1).as_numpy_iterator()),
                             freeze_layers_up_to=3)

    # Workaraound for saving locally when using cloud TPUs
    local_device_option = tf.train.CheckpointOptions(
        experimental_io_device="/job:localhost")
else:
    # GPUs and local systems don't need the above specifications. We simply
    # create a pattern for the filename and let the callbacks deal with it.
    checkpoint_path = os.path.join(checkpoint_dir, "cp-{epoch:04d}.ckpt")
    # Also, on TPU we cannot use tensorboard, but on GPU we can
    log_dir = os.path.join(ROOT_PATH, "data", "logs", 
        "training_dpr", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    
    model = create_model(sample=next(dataset_train.take(1).as_numpy_iterator()),
                             freeze_layers_up_to=3)

    if DO_TRAINING:
        # ModelCheckpoint callback is only available when not using TPU
        cp_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath = checkpoint_path,
            verbose=1,
            save_weights_only = True,
            save_best_only = False
        )

        # Same for tensorboard callback
        tensorboard_callback = tf.keras.callbacks.TensorBoard(
            log_dir=log_dir,
            histogram_freq=1
        )

if DO_TRAINING:
    # Early stopping can be used by both hardware
    es_callback = tf.keras.callbacks.EarlyStopping(
        patience = PATIENCE,
        restore_best_weights=True
    )

    if using_TPU:
        # Save first weights in a h5 file (it's the most stable way)
        model.save_weights(os.path.join(
            checkpoint_dir, 'training_normal_tpu_0.h5'),  overwrite=True)
    else:
        # Save the first weights using the pattern from before
        model.save_weights(checkpoint_path.format(epoch=0))

    callbacks = [es_callback]
    if not using_TPU:
        # These callback imply saving stuff on local disk, which cannot be 
        # done automatically using TPUs.
        # Therefore, they are only active when using GPUs and local systems
        callbacks.extend([cp_callback, tensorboard_callback])

    # We fit the model
    history = model.fit(
        dataset_train, 
        y=None,
        validation_data=dataset_val,
        epochs=EPOCHS, 
        callbacks=callbacks,
        shuffle=True,
        use_multiprocessing=True,
        initial_epoch=0,
        verbose=1 # Show progress bar
    )

    if using_TPU:
        # Save last weights
        model.save_weights(os.path.join(
            checkpoint_dir, 'training_normal_tpu_last.h5'), overwrite=True)

Creating BERT models...


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_projector', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.
Some layers from the model checkpoint at distilbert-base-uncased were no

Creating Deep Encoder...
Compiling...
Testing on some data...
Model created!


Note: the model was trained in 22 epochs on a cloud TPU. It took about two hours and reached a validation accuracy of 99.3%.

# Question Answering with the DPR

We have trained the two Bert (`bert_p`, the paragraphs encoder and `bert_q`, the questions encoder) models to produce embeddings that are as similar as possible for matching question-paragraph pairs. 

Thanks to our training, when we use `bert_q` to encode our question, we will now be sure that questions and paragraphs will both be encoded in the same space and have a high similarity between the question encoding and the matching paragraph's encoding.

We can now use `bert_p` to encode all of our paragraphs a-priori using the same method we have used before (taking the 768-d encoding at the `[CLS]` token). These encodings will be stored in RAM. 

Then, we can define our final Question Answering model in this way:
- It receives only a question's embedding as input.
- It uses `bert_q` to create a representation of the question in the learnt 768-d space, that is in common with the paragraph representations.
- We compute similarity scores between the representation of the question and all representations of paragraphs. Based on these scores, we select the top-k ($k=100$) paragraphs.
- For each of the $k$ paragraphs, we must compute the probability of the paragraph being selected $P_{selected}(i)$, as well as the usual $P_{start, i}(s), P_{end, i}(t)$ for each of the $s$-th and $t$-th words of the $i$-th paragraph. To do that, we need the full encoding of the paragraph (the $512 \times 768$ output of Bert), which will be denoted as $P_i$ in contrast to $\hat{P}_i$ which is the 768-d encoding at the `[CLS]` token. We obtain the full encoding by passing the $k$ paragraphs through `bert_p`, which is set to non-trainable (otherwise the encoding of the `[CLS]` token would constantly change). 
- All probabilities are computed through dense layers:
\begin{gather}
P_{start,i}(s) = softmax(P_i w_{start})_s
\\
P_{end,i}(t) = softmax(P_i w_{end})_t
\\
P_{selected}(i) = softmax(\hat{P}^\intercal w_{selected})_i
\\
\end{gather}
where $w_{start}$, $w_{end}$ and $w_{selected}$ are learnt vectors, while $\hat{P} = [P_{1}^{[CLS]}, \dots, P_k^{[CLS]}]$.
- As final answer, we select the highest scoring start-end legal span from the highest-scoring paragraph.

During training: For each question, we create a batch by sampling $m$ ($m=24$ in the paper) from the top-100 passages returned by the retrieval system (DPR, so by computing similarities with the pre-computed representations). The training objective is to maximize the marginal log-likelihood of all the correct answer spans in the positive passage (the answer string may appear multiple times in one passage), combined with the log-likelihood of the positive passage being selected. In the paper, a batch size of 16 was used.


### Paragraphs representations

First of all, we load the weights of the trained model.

In [18]:
# Load the obtained weights
model.load_weights(os.path.join(checkpoint_dir, 'training_normal_tpu_last.h5'))

Then, we allocate space for the matrices that will contain all paragraphs encoding.

In [19]:
train_paragraphs_encodings = np.empty(
    shape=(len(train_paragraphs), BERT_DIMENSIONALITY)
)
val_paragraphs_encodings = np.empty(
    shape=(len(val_paragraphs), BERT_DIMENSIONALITY)
)
print(train_paragraphs_encodings.shape, val_paragraphs_encodings.shape)

(18896, 768) (4921, 768)


In [20]:
if not os.path.exists(os.path.join(checkpoint_dir, 'train_paragraphs_encodings.txt')) or \
   OVERRIDE_REPRESENTATIONS:

    print("Obtaining training representations")
    for i in tqdm(range(len(train_paragraphs))):
        train_paragraphs_encodings[i] = model.enc.model_p(tokenizer_distilbert(
            train_paragraphs[i]['context'], max_length = 512, 
            return_tensors='tf',
            truncation = True, padding = 'max_length'
        )).last_hidden_state[0,0,:]
    
    np.savetxt(os.path.join(checkpoint_dir, 'train_paragraphs_encodings.txt'), 
               train_paragraphs_encodings, delimiter=',')

if not os.path.exists(os.path.join(checkpoint_dir, 'val_paragraphs_encodings.txt')) or \
   OVERRIDE_REPRESENTATIONS:

    print("Obtaining validation representations")
    for i in tqdm(range(len(val_paragraphs))):
        val_paragraphs_encodings[i] = model.enc.model_p(tokenizer_distilbert(
            val_paragraphs[i]['context'], max_length = 512, 
            return_tensors='tf',
            truncation = True, padding = 'max_length'
        )).last_hidden_state[0,0,:]

    np.savetxt(os.path.join(checkpoint_dir, 'val_paragraphs_encodings.txt'),
               val_paragraphs_encodings, delimiter=',')

In [21]:
train_paragraphs_encoding = np.loadtxt(os.path.join(checkpoint_dir, 
                                                    'train_paragraphs_encodings.txt'), 
                                       delimiter=',', dtype=np.float32)
val_paragraphs_encoding = np.loadtxt(os.path.join(checkpoint_dir, 
                                                  'val_paragraphs_encodings.txt'), 
                                     delimiter=',', dtype=np.float32)

### Qualitative test

We select some random questions and see whether the selected paragraph is the correct one.

In [22]:
sample_question = random.choice(train_questions)
print(f"Question: {sample_question['qas']['question']}")
print(f"Ground truth paragraph: {get_paragraph_from_question(sample_question, train_dict)['context']}")
print(f"Ground truth index: {get_paragraph_global_id_from_question(sample_question, train_paragraphs)}")
sample_q_repr = model.enc.model_q(tokenizer_distilbert(
    sample_question['qas']['question'], max_length = 512, 
    return_tensors='tf', truncation = True, padding = 'max_length'
)).last_hidden_state[:,0,:]
print(f"Question representation shape: {sample_q_repr.shape}")
scores = tf.tensordot(sample_q_repr, train_paragraphs_encoding.T, axes=1)
best_par_index = tf.argsort(scores, axis=1, direction='DESCENDING')[0, :5].numpy()
print(f"Top-5 best matching paragraphs have indexes {best_par_index}")
print(f"Top-5 best matching paragraphs:")
for i in range(5):
    print(f"{i})\t{train_paragraphs[best_par_index[i]]['context']}")

Question: What poet wrote a long poem describing Roman religious holidays?
Ground truth paragraph: The meaning and origin of many archaic festivals baffled even Rome's intellectual elite, but the more obscure they were, the greater the opportunity for reinvention and reinterpretation — a fact lost neither on Augustus in his program of religious reform, which often cloaked autocratic innovation, nor on his only rival as mythmaker of the era, Ovid. In his Fasti, a long-form poem covering Roman holidays from January to June, Ovid presents a unique look at Roman antiquarian lore, popular customs, and religious practice that is by turns imaginative, entertaining, high-minded, and scurrilous; not a priestly account, despite the speaker's pose as a vates or inspired poet-prophet, but a work of description, imagination and poetic etymology that reflects the broad humor and burlesque spirit of such venerable festivals as the Saturnalia, Consualia, and feast of Anna Perenna on the Ides of March,

Usually, the correct paragraphs show up in the Top-5 list.

### Quantitative test

We can measure the top-1/top-5 accuracy with the same used in the tf-idf baseline.

In [23]:
count_top1 = 0
count_top5 = 0
count_total = len(val_questions)

if RUN_ACCURACY_ANALYSIS:
    def top_5_for_question(question):
        sample_q_repr = model.enc.model_q(question).last_hidden_state[:,0,:]
        scores = tf.tensordot(sample_q_repr, val_paragraphs_encoding.T, axes=1)
        top5_indices = tf.argsort(scores, axis=1, direction='DESCENDING')[0, :5].numpy()
        top5_scores = tf.sort(scores, axis=1, direction='DESCENDING')[0, :5].numpy()
        top5_para = [val_paragraphs[i] for i in top5_indices]
        return top5_para, top5_scores, top5_indices     

    for i,q in enumerate(tqdm(val_questions)):
        top5_para, top5_scores, top5_indices = top_5_for_question(
            tokenizer_distilbert(
                q['qas']['question'], max_length = 512, 
                return_tensors='tf', truncation = True, padding = 'max_length'
            )
        )
        top5_context_ids = [top5_para[i]['context_id'] for i in range(len(top5_para))]
        gt_context_id = q['context_id'][0]
        if gt_context_id == top5_context_ids[0]:
            count_top1 += 1
        if gt_context_id in top5_context_ids:
            count_top5 += 1

    top1_score = count_top1 / count_total * 100
    top5_score = count_top5 / count_total * 100

    print(f"\nTop 1 score: {top1_score:.2f}%,\nTop 5 score: {top5_score:.2f}%")

Top 1 score: 86.49%

Top 5 score: 96.58%

This is a clear improvement with respect to the Tf-Idf baseline (72.01%/87.82%), but it's also quite slower. 
- It may look as an unfair comparison, because the Tf-Idf baseline was tested on all questions, but in fact it's more realistic because it's done on a subset of data that the model has not been trained on.

## Question Answering model

In [24]:
class DensePassageRetriever(keras.Model):
    '''
    This model retrieves the best available passages given a question
    and a set of paragraphs (encodings). The model is non-trainable,
    as it assumes that the deep question encoder (model_q) has already been
    trained.
    '''
    def __init__(self, model_q):
        super().__init__()
        self.model_q = model_q

    def call(self, inputs, m=100):
        '''
        Assumes that inputs is a dictionary containing (at least):
        - `input_ids`: the input ids of the questions obtained from the tokenizer
        - `attention_mask`: the attention mask of the questions obtained from the tokenizer
        - `paragraphs_encodings`: the matrix of paragraph encodings for the dataset (num_parag x encoding_dim)

        Returns the (batch_size x num_parag) matrix of scores and the (batch_size x m) top_m indexes (according
        to parameter `m` which defaults to 100)
        '''
        # 1) Obtain search encoding of question
        qs = {
            'input_ids': inputs['questions']['input_ids'],
            'attention_mask': inputs['questions']['attention_mask']
        }
        q_repr = self.model_q(qs).last_hidden_state[:,0,:]                                   # batch_size x encoding_dim
        # 2) Selection of paragraphs using search encoding
        scores = tf.tensordot(q_repr, tf.transpose(inputs['paragraphs_encodings']), axes=1)  # batch_size x num_parag
        topm_indexes = tf.argsort(scores, axis=1, direction='DESCENDING')[:, :m]             # batch_size x m
        return scores, topm_indexes


class QuestionAnsweringModel(keras.Model):

    def __init__(self, model_q, model_p, tokenizer, train_m = 24, m = 50):
        super().__init__()
        self.dpr = DensePassageRetriever(model_q)
        self.model_p = model_p
        self.tokenizer = tokenizer
        # The dot product + softmaxes are implemented as Dense layers
        # with only 1 output and a later softmax activation
        self.dense_s = keras.layers.TimeDistributed(keras.layers.Dense(1), name="start_token_probability_layer") # Output: n_par_selectedx512x1 scores
        self.dense_e = keras.layers.TimeDistributed(keras.layers.Dense(1), name="end_token_probability_layer")   # Output: n_par_selectedx512x1 scores
        self.dense_sel = keras.layers.Dense(1, name="paragraph_selection_probability_layer")                     # Output: n_par_selectedx1 scores
        self.flatten = keras.layers.TimeDistributed(keras.layers.Flatten(), name="flatten_layer")                # Flattening utility layer
        # We always train on the same data so we save the training encodings
        self.train_encoding_dataset = train_paragraphs_encoding
        self.train_paragraphs = tf.stack([train_paragraphs[i]['context'] for i in range(len(train_paragraphs))])
        # The encodings and paragraphs of the other set (validation? test?) should be
        # loaded in before using the model
        self.other_encoding_dataset = None
        self.other_paragraphs = None
        # Set the dpr, model_q and model_p as non-trainable
        for l in self.dpr.model_q.layers:
            l.trainable = False
        for l in self.dpr.layers:
            l.trainable = False
        for l in self.model_p.layers:
            l.trainable = False
        # Other parameters
        self.train_m = train_m
        self.m = m


    def load_testing_set_of_paragraphs(self, paragraphs, paragraphs_encoding):
        # REMEMBER: switch the dataset of paragraphs encoding
        # when moving from validation to test
        self.other_paragraphs = np.stack([paragraphs[i]['context'] for i in range(len(paragraphs))])
        self.other_encoding_dataset = paragraphs_encoding


    def collect_paragraphs_from_indices(self, data_batch, paragraphs):
        # Collect paragraphs
        paragraphs = tf.gather(paragraphs, data_batch).numpy()
        # Tokenize the selected paragraphs
        tokenized_paragraphs = [dict(self.tokenizer(
            [par.decode('utf-8') for par in paragraphs[p]], max_length = 512, 
            return_tensors='tf', truncation = True, 
            padding = 'max_length', return_offsets_mapping = True
        )) for p in range(len(paragraphs))]
        # Compute full encoding (cannot pre-encode because it takes too much space) (also, for test mode)
        paragraphs_full_encodings = [self.model_p({
                'input_ids': tokenized_paragraphs[p]['input_ids'],
                'attention_mask': tokenized_paragraphs[p]['attention_mask']
            }).last_hidden_state for p in range(len(paragraphs))]                   # batch_size x (mx512x768) tensors
        # The search encoding is simply the first token of these
        paragraphs_search_encodings = [p[:,0,:] for p in paragraphs_full_encodings] # batch_size x (mx768) tensors

        return paragraphs, tokenized_paragraphs, \
            tf.stack(paragraphs_full_encodings), \
            tf.stack(paragraphs_search_encodings)

    def call(self, inputs, training=False):
        if not training:
            # inputs should be simply be the question string
            tokenized_q = dict(self.tokenizer(
                inputs[0], max_length = 512, 
                return_tensors='tf', truncation = True, 
                padding = 'max_length', return_offsets_mapping = True
            ))

            # Do what should be done in the test step:

            # Obtain the topm_indexes for the question
            _, topm_indexes = self.dpr({
                'questions': {
                    'input_ids': tokenized_q['input_ids'],
                    'attention_mask': tokenized_q['attention_mask']
                },
                'paragraphs_encodings': self.other_encoding_dataset
            }, m = self.m)
            
            # In the test case, the data batch is simply made of the returned paragraphs
            data_batch = topm_indexes

            # Gather paragraphs and compute their encodings
            paragraphs, tokenized_paragraphs, paragraphs_full_encodings, paragraphs_search_encodings = \
                self.collect_paragraphs_from_indices(data_batch, self.other_paragraphs)

            # Create the input dictionary
            inputs = {
                'paragraphs_full_encodings': paragraphs_full_encodings,
                'paragraphs_search_encodings': paragraphs_search_encodings
            }

        # Probabilities calculations
        probs_s = tf.nn.softmax(self.flatten(self.dense_s(inputs['paragraphs_full_encodings'])), axis=2)
        probs_e = tf.nn.softmax(self.flatten(self.dense_e(inputs['paragraphs_full_encodings'])), axis=2)
        probs_sel = tf.nn.softmax(self.dense_sel(inputs['paragraphs_search_encodings']), axis=1)

        # Selection of best scoring paragraphs
        best_scoring_paragraphs = tf.cast(tf.squeeze(tf.argmax(probs_sel, axis=1)), tf.int32)

        # Selection of related start-end probabilities
        probs_s = tf.squeeze(tf.gather(probs_s, indices=tf.expand_dims(best_scoring_paragraphs, -1), batch_dims=1))
        probs_e = tf.squeeze(tf.gather(probs_e, indices=tf.expand_dims(best_scoring_paragraphs, -1), batch_dims=1))

        if training:
            # Compute losses
            self.add_loss(keras.losses.categorical_crossentropy(
                y_true=inputs['out_s'], y_pred=probs_s
            ))
            self.add_loss(keras.losses.categorical_crossentropy(
                y_true=inputs['out_e'], y_pred=probs_e
            ))
            self.add_loss(keras.losses.categorical_crossentropy(
                y_true=inputs['gt_mask'], y_pred=tf.squeeze(probs_sel)
            ))

            # Update metrics
            self.add_metric(keras.metrics.categorical_accuracy(
                y_true=inputs['out_s'], y_pred=probs_s), 
                name="start_token_accuracy")
            self.add_metric(keras.metrics.categorical_accuracy(
                y_true=inputs['out_e'], y_pred=probs_e), 
                name="end_token_accuracy")
            self.add_metric(keras.metrics.categorical_accuracy(
                y_true=inputs['gt_mask'], y_pred=tf.squeeze(probs_sel)), 
                name="selection_accuracy")

            return probs_sel, probs_s, probs_e
        
        else:

            return paragraphs, tokenized_paragraphs, best_scoring_paragraphs, \
                   probs_sel, probs_s, probs_e


    @tf.function
    def obtain_training_info(self, indexes, topm_indexes):
        # Collect ground truth indexes
        gt_paragraphs = tf.expand_dims(tf.cast(indexes, tf.int32), -1)
        # A training sample is formed by the positive and m-1 negative examples
        # obtained from the top-100 for each of the questions in the batch.
        # We create a data batch by sampling m-1 examples from the masked 100 paragraphs
        negative_masks = tf.math.not_equal(topm_indexes, gt_paragraphs)
        # To keep the graph working with the correct sizes, we create a tensor of negatives 
        # by random shuffling the large tensor of topm and taking the first train_m elements.
        # The positive examples are replaced by randomly sampling from the same tensor.
        # It could happen that the positive example is replaced by itself, or that a 
        # negative sample appears twice in the batch, but it's a non-deterministic
        # process.
        negatives = self.tf_shuffle_on_columns(tf.where(
            negative_masks, topm_indexes, self.tf_shuffle_on_columns(topm_indexes))
        )[:,:self.train_m-1]
        # We concatenate the positive paragraph index to the selected negatives and shuffle
        # so that the positive is not always the last element
        data_batch = self.tf_shuffle_on_columns(
            tf.concat([negatives, gt_paragraphs], axis=1))
        # When we have a data batch, we create the ground truth mask, which represents the position
        # of the positive sample in the data batch in a one-hot encoded fashion.
        gt_mask = tf.cast(data_batch == gt_paragraphs, tf.int32)
        return data_batch, gt_mask


    @tf.function
    def tf_shuffle_on_columns(self, value):
        # Create a tensor of random numbers, argsort it and use them as indices to gather
        # values from the original tensor
        return tf.gather(value, tf.argsort(tf.random.uniform(tf.shape(value))), batch_dims=1)


    def train_step(self, data):
        # Obtain scores and paragraphs top indexes
        _, topm_indexes = self.dpr({
            'questions': {
                'input_ids': data['questions']['input_ids'],
                'attention_mask': data['questions']['attention_mask']
            },
            'paragraphs_encodings': self.train_encoding_dataset
        })
        
        data_batch, gt_mask = self.obtain_training_info(
            data['paragraphs']['index'], topm_indexes)

        # Gather paragraphs and compute their encodings
        _, _, paragraphs_full_encodings, paragraphs_search_encodings = \
            self.collect_paragraphs_from_indices(data_batch, self.train_paragraphs)

        # Create the input dictionary
        inputs = {
            'paragraphs_full_encodings': paragraphs_full_encodings,
            'paragraphs_search_encodings': paragraphs_search_encodings,
            'out_s': data['answers']['out_s'],
            'out_e': data['answers']['out_e'],
            'gt_mask': gt_mask
        }
        
        # Activate the gradient tape and collect the output from the model
        with tf.GradientTape() as tape:
            # Obtain probabilities from the model
            probs_sel, probs_s, probs_e = self(inputs, training=True)
            # Obtain loss value
            loss = sum(self.losses)
        
        # Construct gradients and apply them through the optimizer
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        # Return the dict of metrics
        return {m.name: m.result() for m in self.metrics}


    def test_step(self, data):
        # Obtain scores and paragraphs top indexes
        _, topm_indexes = self.dpr({
            'questions': {
                'input_ids': data['questions']['input_ids'],
                'attention_mask': data['questions']['attention_mask']
            },
            'paragraphs_encodings': self.other_encoding_dataset
        }, m = self.m)
        
        # In the test case, the data batch is simply made of the returned paragraphs
        data_batch = topm_indexes

        # Gather paragraphs and compute their encodings
        _, _, paragraphs_full_encodings, paragraphs_search_encodings = \
            self.collect_paragraphs_from_indices(data_batch, self.other_paragraphs)

        # Create the input dictionary
        inputs = {
            'paragraphs_full_encodings': paragraphs_full_encodings,
            'paragraphs_search_encodings': paragraphs_search_encodings,
            'out_s': data['answers']['out_s'],
            'out_e': data['answers']['out_e'],
            'gt_mask': gt_mask
        }

        # Obtain the probabilities from the model and return the dict of metrics
        probs_sel, probs_s, probs_e = self(inputs, training=True)
        return {m.name: m.result() for m in self.metrics}


    def start_end_token_from_probabilities(
        pstartv: np.array, 
        pendv: np.array, 
        dim:int=512) -> List[List[int]]:
        '''
        Returns a List of [StartToken, EndToken] elements computed from the batch outputs.
        '''
        idxs = []
        for i in range(pstartv.shape[0]):
            # For each element in the batch, transform the vectors into matrices
            # by repeating them dim times:
            # - Vectors of starting probabilities are stacked on the columns
            pstart = np.stack([pstartv[i,:]]*dim, axis=1)
            # - Vectors of ending probabilities are repeated on the rows
            pend = np.stack([pendv[i,:]]*dim, axis=0)
            # Once we have the two matrices, we sum them (element-wise operation)
            # to obtain the scores of each combination
            sums = pstart + pend
            # We only care about the scores in the upper triangular part of the matrix
            # (where the ending index is greater than the starting index)
            # therefore we zero out the diagonal and the lower triangular area
            sums = np.triu(sums, k=1)
            # The most probable set of tokens is the one with highest score in the
            # remaining matrix. Through argmax we obtain its position.
            val = np.argmax(sums)
            # Since the starting probabilities are repeated on the columns, each element
            # is identified by the row. Ending probabilities are instead repeated on rows,
            # so each element is identified by the column.
            row = val // dim
            col = val - dim*row
            idxs.append([row,col])
        return idxs


def create_model_qa(model_q, model_p, tokenizer, sample):
    print("Creating Question Answering model...")
    model = QuestionAnsweringModel(model_q, model_p, tokenizer)

    print("Compiling...")
    # Compile the model (loss and metrics are defined into the model)
    model.compile(
        run_eagerly=True,
        optimizer=keras.optimizers.Adam(learning_rate=3e-6)
    )

    # Load the validation set
    model.load_testing_set_of_paragraphs(val_paragraphs, 
                                         val_paragraphs_encoding)

    print("Testing on some data...")
    # Pass one batch of data to build the model
    model(sample, training=False)

    # Visualize the model's structure
    model.summary()

    # Return the model
    print("Model created!")
    return model

Train the model:

In [ ]:
import datetime

if using_TPU:
    # TPU requires to create the model within the scope of the distributed strategy
    # we're using.
    with strategy.scope():
        model_qa = create_model_qa(model.enc.model_q, model.enc.model_p,tokenizer_distilbert,
                                sample="Who are you?")

    # Workaraound for saving locally when using cloud TPUs
    local_device_option = tf.train.CheckpointOptions(
        experimental_io_device="/job:localhost")
else:
    # GPUs and local systems don't need the above specifications. We simply
    # create a pattern for the filename and let the callbacks deal with it.
    checkpoint_path = os.path.join(checkpoint_dir, "cp-{epoch:04d}.ckpt")
    # Also, on TPU we cannot use tensorboard, but on GPU we can
    log_dir = os.path.join(ROOT_PATH, "data", "logs", 
        "training_qa_dpr", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    
    model_qa = create_model_qa(model.enc.model_q, model.enc.model_p, tokenizer_distilbert,
                            sample="Who are you?")

    if DO_QA_TRAINING:
        # ModelCheckpoint callback is only available when not using TPU
        cp_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath = checkpoint_path,
            verbose=1,
            save_weights_only = True,
            save_best_only = False
        )

        # Same for tensorboard callback
        tensorboard_callback = tf.keras.callbacks.TensorBoard(
            log_dir=log_dir,
            histogram_freq=1
        )

if DO_QA_TRAINING:
    # Early stopping can be used by both hardware
    es_callback = tf.keras.callbacks.EarlyStopping(
        patience = PATIENCE,
        restore_best_weights=True
    )

    if using_TPU:
        # Save first weights in a h5 file (it's the most stable way)
        model_qa.save_weights(os.path.join(
            checkpoint_dir, 'training_qa_tpu_0.h5'), overwrite=True)
    else:
        # Save the first weights using the pattern from before
        model_qa.save_weights(checkpoint_path.format(epoch=0))

    callbacks = [es_callback]
    if not using_TPU:
        # These callback imply saving stuff on local disk, which cannot be 
        # done automatically using TPUs.
        # Therefore, they are only active when using GPUs and local systems
        callbacks.extend([cp_callback, tensorboard_callback])

    # We fit the model
    history = model_qa.fit(
        dataset_train, 
        y=None,
        validation_data=dataset_val,
        epochs=EPOCHS, 
        callbacks=callbacks,
        shuffle=True,
        use_multiprocessing=True,
        initial_epoch=0,
        verbose=1 # Show progress bar
    )

    if using_TPU:
        # Save last weights
        model_qa.save_weights(os.path.join(
            checkpoint_dir, 'training_qa_tpu_last.h5'), overwrite=True)

## Tests

First, we make a test to check if the pipeline is working fine.

In [78]:
inputs = next(dataset_val.take(1).as_numpy_iterator())

scores, top100_indexes = model_qa.dpr({
    'questions': {
        'input_ids': inputs['questions']['input_ids'],
        'attention_mask': inputs['questions']['attention_mask']
    },
    'paragraphs_encodings': val_paragraphs_encoding
})

In [79]:
top100_indexes[:,:5]

<tf.Tensor: shape=(8, 5), dtype=int32, numpy=
array([[1203, 1216, 1217, 1219, 2118],
       [ 190,  235,  238,  205,  241],
       [1489, 1476, 1468, 1475, 1483],
       [  49,   55,   48,   56,   50],
       [1977, 1979, 1991, 1960, 1975],
       [1998, 1909, 1994, 1999, 1937],
       [ 889, 3878,  890,  883,  888],
       [1904, 1986, 1955, 1954, 1957]], dtype=int32)>

In [80]:
model_qa.obtain_training_info(inputs['paragraphs']['index'], top100_indexes)

(<tf.Tensor: shape=(8, 24), dtype=int32, numpy=
 array([[1225,  326, 2150, 1223, 2174, 1213, 4186, 2129, 3251, 1224, 1217,
         1918, 2148, 2134, 1950, 2142, 2144, 1215, 1231, 2156, 2132, 2180,
         1228, 1203],
        [4274,  242, 1247,  232, 1237, 3543, 3753,  205, 1240, 1514,  211,
          713,  734, 1901,  224,  206,  240,  243,  236,  237,  189,  203,
          239,  233],
        [4482,   93, 1486, 1549,  750, 1415, 1051, 4197, 1056, 4211, 1477,
         1493, 1467, 1463,   78, 1488, 3148,  793,   80,   74, 1489, 1887,
         1487, 1481],
        [  58,    3,   38,   64,   43, 3235, 3109,    8, 3944, 3930,   49,
         2577,  357,   57,   50, 1615,   60, 4774, 4552,   59, 3947,   44,
         1085, 1619],
        [1958, 1947,  232, 3047, 1774, 3254, 2765, 3295, 2741, 4138, 1025,
         2744, 2744, 1028, 1991, 1950, 3300, 1315, 1949,  251, 2766,  230,
         2767, 2667],
        [1948, 1991, 1954, 1990, 3477, 1950, 1899, 1907, 1934, 1966, 1981,
         2385, 19

In [81]:
gt_paragraphs = tf.expand_dims(tf.cast(inputs['paragraphs']['index'], tf.int32), -1)
gt_paragraphs

<tf.Tensor: shape=(8, 1), dtype=int32, numpy=
array([[1203],
       [ 233],
       [1489],
       [  49],
       [1991],
       [1904],
       [ 888],
       [1904]], dtype=int32)>

In [82]:
gt_paragraphs = tf.cast(inputs['paragraphs']['index'], tf.int32) 
print(top100_indexes.shape, gt_paragraphs.shape)

(8, 100) (8,)


In [83]:
gt_paragraphs

<tf.Tensor: shape=(8,), dtype=int32, numpy=array([1203,  233, 1489,   49, 1991, 1904,  888, 1904], dtype=int32)>

In [84]:
negatives_mask = tf.math.not_equal(top100_indexes, tf.expand_dims(gt_paragraphs, -1))
negatives_mask.shape

TensorShape([8, 100])

In [85]:
negatives = model_qa.tf_shuffle_on_columns(tf.where(negatives_mask, top100_indexes, model_qa.tf_shuffle_on_columns(top100_indexes)))[:,:23]
negatives

<tf.Tensor: shape=(8, 23), dtype=int32, numpy=
array([[ 154, 2176, 1213,  114, 1981, 1909, 1206, 4501, 2141, 2173, 2174,
        2151, 2187, 2170, 2180, 1226, 2150, 2142,  867, 2139, 4508, 2160,
        2155],
       [ 229, 1514, 3543, 3715,  218, 1503,  216,  211, 3753, 1507, 1240,
        1504, 1237,  190,  198, 1262, 3752, 1248,  222,  221,  213, 3717,
        1502],
       [4148, 1053, 1051, 3090, 1471,  758, 1466, 1887, 1549, 1464, 2873,
        2874, 1488, 1474, 1469, 1065, 1463, 1054, 4193, 4174, 2654, 4228,
        1486],
       [  48, 2576, 2557, 2297,   34,   38, 4771,   36, 4540, 3942, 1935,
        2506, 2580,   47,   52, 4851, 4794,   46, 3105,    7,  572, 2578,
        2575],
       [2516,  228, 1970, 3340, 2061, 1981, 4785, 1988, 1950,  230, 1034,
         227, 2767, 2782, 2740, 4019, 2743, 1960, 2739,  251, 2764, 1953,
        4812],
       [4271, 1969, 1999, 3395, 4578, 3468, 3338, 1902, 1934, 1900, 1937,
        3459, 1992, 1909, 3386, 4400, 1903, 1917, 1907, 1986, 19

In [86]:
data_batch = model_qa.tf_shuffle_on_columns(tf.concat([negatives, tf.expand_dims(gt_paragraphs, -1)], axis=1))
data_batch

<tf.Tensor: shape=(8, 24), dtype=int32, numpy=
array([[2160, 1909, 2173, 1981,  114, 4501, 2155, 2176, 2142,  154, 2187,
        1206, 4508, 2150, 1226,  867, 1203, 1213, 2170, 2141, 2151, 2139,
        2174, 2180],
       [ 218,  222, 1262, 1237, 1514, 3753,  213, 3717, 1507, 3752, 1240,
        1502,  211, 1503, 1504,  216, 3543, 3715,  221,  233, 1248,  190,
         198,  229],
       [1549, 1053, 1489, 1474, 1488, 3090, 1887, 1466, 1463, 1469, 4148,
        1471, 1486, 4193, 4174, 4228, 1464, 1065, 1054, 2654, 1051, 2874,
        2873,  758],
       [4540,   48,   34, 2576,   52,   47,   38, 4794,   46, 2575, 2578,
        2297, 1935, 3105,  572, 2506,    7, 2580, 3942,   49,   36, 4771,
        4851, 2557],
       [2782, 2767, 1981,  230, 4812, 2740, 2764, 2743, 1950, 3340,  228,
        1960, 2516, 4019,  251, 1034, 1953, 1970, 2739, 4785, 1988,  227,
        2061, 1991],
       [1969, 1902, 1909, 1937, 3386, 1992, 4271, 1903, 1999, 1986, 4578,
        3395, 1990, 1934, 3338, 44

In [87]:
gt_mask = tf.cast(data_batch == tf.expand_dims(gt_paragraphs, -1), tf.int32)
gt_mask

<tf.Tensor: shape=(8, 24), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0]], dtype=int32)>

In [88]:
class ParagraphCollectorLayer(keras.layers.Layer):
    
    def __init__(self, model_p, tokenizer, **kwargs):
        # This layer is executed in eager mode, because it must
        # use the values of the indices inside the data batch tensor
        # to gather paragraphs
        super().__init__(dynamic=True, trainable=False, **kwargs)
        self.model_p = model_p
        self.tokenizer = tokenizer
    
    def call(self, inputs):
        # Inputs should contain the tensor of indices [0] and the list of paragraphs [1]
        # (train or other)
        data_batch = inputs[0]
        paragraphs = inputs[1]
        # Collect paragraphs
        paragraphs = tf.gather(paragraphs, data_batch).numpy()
        # Tokenize the selected paragraphs
        tokenized_paragraphs = [dict(self.tokenizer(
            [par.decode('utf-8') for par in paragraphs[p]], max_length = 512, 
            return_tensors='tf', truncation = True, 
            padding = 'max_length', return_offsets_mapping = True
        )) for p in range(len(paragraphs))]
        # Compute full encoding (cannot pre-encode because it takes too much space)
        paragraphs_full_encodings = [self.model_p({
                'input_ids': tokenized_paragraphs[p]['input_ids'],
                'attention_mask': tokenized_paragraphs[p]['attention_mask']
            }).last_hidden_state for p in range(len(paragraphs))]                   # batch_size x (mx512x768) tensors
        # The search encoding is simply the first token of these
        paragraphs_search_encodings = [p[:,0,:] for p in paragraphs_full_encodings] # batch_size x (mx768) tensors

        return tf.stack(paragraphs_full_encodings), tf.stack(paragraphs_search_encodings)

In [89]:
par_coll_layer = ParagraphCollectorLayer(model.enc.model_p, tokenizer_distilbert)

In [90]:
paragraphs_full_encodings, paragraphs_search_encodings = par_coll_layer(
    [data_batch, np.stack([val_paragraphs[i]['context'] for i in range(len(val_paragraphs))])])

In [91]:
paragraphs_full_encodings.shape

TensorShape([8, 24, 512, 768])

In [92]:
paragraphs_search_encodings.shape

TensorShape([8, 24, 768])

In [93]:
# Layers definitions
dense_s = keras.layers.TimeDistributed(keras.layers.Dense(1)) # Output: 24x512x1 scores
dense_e = keras.layers.TimeDistributed(keras.layers.Dense(1)) # Output: 24x512x1 scores
dense_sel = keras.layers.Dense(1) # Output: 24x1 scores
flatten = keras.layers.TimeDistributed(keras.layers.Flatten())
softmax = keras.layers.Softmax(axis=1)

In [94]:
probs_s = tf.nn.softmax(flatten(dense_s(paragraphs_full_encodings)), axis=2)
probs_e = tf.nn.softmax(flatten(dense_e(paragraphs_full_encodings)), axis=2)
probs_sel = tf.nn.softmax(dense_sel(paragraphs_search_encodings), axis=1)
print(probs_s.shape, probs_e.shape, probs_sel.shape)

(8, 24, 512) (8, 24, 512) (8, 24, 1)


In [95]:
# Selection of best scoring paragraphs
best_scoring_paragraphs = tf.squeeze(tf.argmax(probs_sel, axis=1))
best_scoring_paragraphs

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([20,  7, 19,  7, 12, 13, 20, 18])>

In [96]:
# Selection of related start-end probabilities
probs_s = tf.squeeze(tf.gather(probs_s, indices=tf.expand_dims(best_scoring_paragraphs, -1), batch_dims=1))
probs_e = tf.squeeze(tf.gather(probs_e, indices=tf.expand_dims(best_scoring_paragraphs, -1), batch_dims=1))

In [97]:
print(best_scoring_paragraphs.shape, probs_s.shape, probs_e.shape)

(8,) (8, 512) (8, 512)


In [98]:
## Loss functions:
# 1) Probability loss
loss = tf.keras.losses.CategoricalCrossentropy()
answer_s_loss = loss(y_true=inputs['answers']['out_s'], y_pred=probs_s)
answer_e_loss = loss(y_true=inputs['answers']['out_e'], y_pred=probs_e)
# 2) Selection loss
selection_loss = loss(y_true=tf.stack(gt_mask), y_pred=tf.squeeze(probs_sel))

print(f"Answer start loss: {answer_s_loss.numpy()}, Answer end loss: {answer_e_loss.numpy()}, Selection loss: {selection_loss.numpy()}")

Answer start loss: 6.138369083404541, Answer end loss: 6.211648941040039, Selection loss: 3.3223214149475098


We reuse the function that returns the start and end tokens of the question given the computed probabilities

In [ ]:
def start_end_token_from_probabilities(
    pstartv: np.array, 
    pendv: np.array, 
    dim:int=512) -> List[List[int]]:
    '''
    Returns a List of [StartToken, EndToken] elements computed from the batch outputs.
    '''
    idxs = []
    for i in range(pstartv.shape[0]):
        # For each element in the batch, transform the vectors into matrices
        # by repeating them dim times:
        # - Vectors of starting probabilities are stacked on the columns
        pstart = np.stack([pstartv[i,:]]*dim, axis=1)
        # - Vectors of ending probabilities are repeated on the rows
        pend = np.stack([pendv[i,:]]*dim, axis=0)
        # Once we have the two matrices, we sum them (element-wise operation)
        # to obtain the scores of each combination
        sums = pstart + pend
        # We only care about the scores in the upper triangular part of the matrix
        # (where the ending index is greater than the starting index)
        # therefore we zero out the diagonal and the lower triangular area
        sums = np.triu(sums, k=1)
        # The most probable set of tokens is the one with highest score in the
        # remaining matrix. Through argmax we obtain its position.
        val = np.argmax(sums)
        # Since the starting probabilities are repeated on the columns, each element
        # is identified by the row. Ending probabilities are instead repeated on rows,
        # so each element is identified by the column.
        row = val // dim
        col = val - dim*row
        idxs.append([row,col])
    return idxs

In [ ]:
answers_start_end = start_end_token_from_probabilities(probs_s, probs_e)
print(answers_start_end)

[[103, 201], [4, 173], [7, 159], [172, 353], [113, 115], [56, 223], [127, 129], [132, 172]]


Finally, we can obtain the answers to the questions we have given the network.

In [ ]:
char_start_end = [(tokenized_paragraphs[i]['offset_mapping'][best_scoring_paragraphs[i]][answers_start_end[i][0]][0].numpy(),
                   tokenized_paragraphs[i]['offset_mapping'][best_scoring_paragraphs[i]][answers_start_end[i][1]][1].numpy())
                 for i in range(BATCH_SIZE)]
char_start_end

[(499, 0),
 (13, 0),
 (20, 0),
 (813, 0),
 (559, 569),
 (261, 0),
 (604, 614),
 (599, 761)]

In [ ]:
# Correction for answers arriving to the end of the sequence
for i in range(BATCH_SIZE):
    c = char_start_end[i]
    if c[1] >= c[0]:
        char_start_end[i] = (c[0], c[1])
    else:
        char_start_end[i] = (c[0], 1000000)

In [ ]:
answers = [paragraphs[i][best_scoring_paragraphs[i]]['context'][char_start_end[i][0]:char_start_end[i][1]] 
           for i in range(BATCH_SIZE)]
for a in answers:
    print(a)

.
hand, many of the medieval buildings once situated within the town walls are now in ruins or have disappeared altogether. From successive incarnations of the motte and bailey castle, only a section of the bailey wall remains today, lying just off Castle Way. The last remains of the Franciscan friary in Southampton, founded circa 1233 and dissolved in 1538, were swept away in the 1940s. The site is now occupied by Friary House.
or region) of DNA that encodes a functional RNA or protein product, and is the molecular unit of heredity.:Glossary The transmission of genes to an organism's offspring is the basis of the inheritance of phenotypic traits. Most biological traits are under the influence of polygenes (many different genes) as well as the gene–environment interactions. Some genetic traits are instantly visible, such as eye colour or number of limbs, and some are not, such as blood type, risk for specific diseases, or the thousands of basic biochemical processes that comprise life.

Of course, answers are extremely bad because we need to train the Dense layers selecting the start and end tokens, as well as the paragraph selector.